In [3]:
import pandas as pd
import numpy as np

In [6]:
test_df = pd.read_parquet('test.parquet',engine='pyarrow')
train_df = pd.read_parquet('train.parquet',engine='pyarrow')

In [5]:
train_df['S_2'] = pd.to_datetime(train_df['S_2'])
train_df = train_df.set_index(['customer_ID','S_2']).sort_index().reset_index()

In [3]:
test_df['S_2'] = pd.to_datetime(test_df['S_2'])
test_df = test_df.set_index(['customer_ID','S_2']).sort_index().reset_index()

In [7]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
num_cols = [x for x in train_df.columns if (x not in cat_features) & (x not in ['customer_ID','S_2'])]

In [6]:
def last_2(arr):
    return arr.values[-2] if len(arr) >= 2 else np.nan
def last_3(arr):
    return arr.values[-3] if len(arr) >= 3 else np.nan

# Train lag

In [7]:
train_num_cols_agg_lag = train_df.groupby('customer_ID')[num_cols].agg([last_2,last_3])
train_num_cols_agg_lag.columns = ['_'.join(x) for x in train_num_cols_agg_lag.columns]

In [8]:
train_cat_features_agg_lag = train_df.groupby('customer_ID')[cat_features].agg([last_2,last_3])
train_cat_features_agg_lag.columns = ['_'.join(x) for x in train_cat_features_agg_lag.columns]

# test lag

In [10]:
test_num_cols_agg_lag = test_df.groupby('customer_ID')[num_cols].agg([last_2,last_3])
test_num_cols_agg_lag.columns = ['_'.join(x) for x in test_num_cols_agg_lag.columns]

In [11]:
test_cat_features_agg_lag = test_df.groupby('customer_ID')[cat_features].agg([last_2,last_3])
test_cat_features_agg_lag.columns = ['_'.join(x) for x in test_cat_features_agg_lag.columns]

# train 合併

In [13]:
train_lag = pd.concat([train_num_cols_agg_lag,train_cat_features_agg_lag],axis=1)

In [14]:
train_lag.to_parquet('train_lag.parquet',index=True)

# test 合併

In [15]:
test_lag = pd.concat([test_num_cols_agg_lag,test_cat_features_agg_lag],axis=1)

In [16]:
test_lag.to_parquet('test_lag.parquet',index=True)

# 

# 做before & after

In [19]:
#train data 最後一筆交易全部集中在2018-03
#test data要分成指交易到2019-4月跟交易到2019-10月的dataset

def before_after_3 (df,startwith,time='2018-03-01'):
    
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    #防止出現兩次S_2
    if startwith == 'S_':
        s = pd.concat([df[['customer_ID']],df.loc[:,df.columns.str.startswith(startwith)]],axis=1)
    else:    
        s = pd.concat([df[['customer_ID','S_2']],df.loc[:,df.columns.str.startswith(startwith)]],axis=1)
    #排除分類的columns
    for col in s.columns:
        if col in cat_features:
            s.drop(col,axis=1,inplace=True)
        
    after_3 = s[s['S_2']>=time].groupby('customer_ID').agg('mean').sort_index()
    before_3 = s[s['S_2']<time].groupby('customer_ID').agg('mean').sort_index()
    if 'S_2' in s.columns:
        s.drop('S_2',axis=1,inplace=True)
    x_s = after_3 - before_3
    x_s.columns = [i+'_before_after'for i in x_s.columns]
    return x_s

In [20]:
firstwith = ['D_','S_','P_','B_','R_']

In [21]:
train_df = pd.read_parquet('train.parquet')
test_df = pd.read_parquet('test.parquet')
train_df.S_2 = pd.to_datetime(train_df.S_2)
test_df.S_2 = pd.to_datetime(test_df.S_2)

# Test

## 先把Test data分成4月跟10月

In [22]:
_=test_df.groupby('customer_ID')['S_2'].nth(-1).dt.month.to_frame()

In [23]:
test_df = test_df.set_index('customer_ID')

In [24]:
#交易紀錄只到4月的
month_4 = _.loc[_['S_2'] == 4,:].index
dataset_month4 = test_df.loc[month_4,:].reset_index()

In [25]:
#交易紀錄到10月的
month_10 = _.loc[_['S_2'] == 10,:].index
dataset_month10 = test_df.loc[month_10,:].reset_index()

## test 4月

In [26]:
#test data的 before-after
month_4_df = pd.DataFrame()
for i in firstwith:
    _ = before_after_3(dataset_month4,i,'2019-04-01')
    month_4_df = pd.concat([month_4_df,_],axis=1)

## test 10月

In [27]:
#test data的 before-after
month_10_df = pd.DataFrame()
for i in firstwith:
    _ = before_after_3(dataset_month10,i,'2019-10-01')
    month_10_df = pd.concat([month_10_df,_],axis=1)

## 4月10月合併

In [28]:
before_after_test = pd.concat([month_4_df,month_10_df],axis=0)

In [29]:
before_after_test.to_parquet('before_after_test.parquet')

# Train

In [30]:
before_after_train  = pd.DataFrame()
for i in firstwith:
    _ = before_after_3(train_df,i,'2018-03-01')
    before_after_train = pd.concat([before_after_train,_],axis=1)

In [31]:
before_after_train.to_parquet('before_after_train.parquet')

# 最後2、3筆跟此前的平均數差異

In [8]:
def diff_with_mean_2(arr):
    return arr.values[-2]-np.mean(arr[:-2])
def diff_with_mean_3(arr):
    return arr.values[-3]-np.mean(arr[:-3])

In [9]:
#train
count = train_df.groupby('customer_ID')['P_2'].count().to_frame()
count.columns = ['count']
train_df=train_df.merge(count,on='customer_ID',how='left')

In [10]:
#test
count = test_df.groupby('customer_ID')['P_2'].count().to_frame()
count.columns = ['count']
test_df=test_df.merge(count,on='customer_ID',how='left')

## 最後第2筆

In [24]:
before_after_train_2 = train_df.loc[train_df['count'] >= 3,:].groupby('customer_ID')[num_cols].apply(diff_with_mean_2)
before_after_train_2.columns = [i+'_before_after_2'for i in before_after_train_2.columns]

In [25]:
before_after_train_2.to_parquet('before_after_train_2.parquet')

In [26]:
before_after_test_2 = test_df.loc[test_df['count'] >= 3,:].groupby('customer_ID')[num_cols].apply(diff_with_mean_2)
before_after_test_2.columns = [i+'_before_after_2'for i in before_after_test_2.columns]

In [27]:
before_after_test_2.to_parquet('before_after_test_2.parquet')

## 最後第3筆

In [11]:
before_after_train_3 = train_df.loc[train_df['count'] >= 4,:].groupby('customer_ID')[num_cols].apply(diff_with_mean_3)
before_after_train_3.columns = [i+'_before_after_3'for i in before_after_train_3.columns]

In [12]:
before_after_test_3 = test_df.loc[test_df['count'] >= 4,:].groupby('customer_ID')[num_cols].apply(diff_with_mean_3)
before_after_test_3.columns = [i+'_before_after_3'for i in before_after_test_3.columns]

In [13]:
before_after_train_3.to_parquet('before_after_train_3.parquet')

In [14]:
before_after_test_3.to_parquet('before_after_test_3.parquet')

# time的features

## train

In [ ]:
df = pd.DataFrame(train_df[['customer_ID','S_2']],columns=['customer_ID','S_2'])
df['S_2'] = pd.to_datetime(df['S_2'])
df['day_count'] = df[['customer_ID','S_2']].groupby('customer_ID')['S_2'].transform('count')
df['S_2_first'] = df[['customer_ID','S_2']].groupby(['customer_ID'])['S_2'].transform('min')
df['S_2_last'] = df[['customer_ID','S_2']].groupby('customer_ID')['S_2'].transform('max')
df['S_2_period'] = (df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('max') - df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('min')).dt.days
df['day_between_statment'] = df[['customer_ID','S_2']].sort_values(['customer_ID','S_2']).groupby(['customer_ID'])['S_2'].transform('diff').dt.days
df['day_between_statment'] = df['day_between_statment'].fillna(0)
df['day_between_statment_mean'] = df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('mean')
df['day_between_statment_std'] =df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('std')
df['day_between_statment_max'] =df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('max')
df['day_between_statment_min'] =df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('min')
#計算每筆交易時間跟最後一筆交易時間間隔多久
df['S_2'] = (df['S_2_last'] - df['S_2']).dt.days 

In [ ]:
#每個人的最後一筆交易日期 距離全部train data最晚的交易資料差距的天數
df['S_2_last_diff'] = (df['S_2_last'].max() - df['S_2_last']).dt.days
#每個人的第一筆交易日期 距離全部train data最早的交易資料差距的天數
df['S_2_first_diff'] = (df['S_2_first'].min()-df['S_2_first']).dt.days
df['S_2_first_dd'] = df['S_2_first'].dt.day
df['S_2_first_mm'] = df['S_2_first'].dt.month
df['S_2_first_yy'] = df['S_2_first'].dt.year
df['S_2_last_dd'] = df['S_2_last'].dt.day
df['S_2_last_mm'] = df['S_2_last'].dt.month
df['S_2_last_yy'] = df['S_2_last'].dt.year

In [ ]:
def last_2(series):
    return series.values[-2] if len(series.values)>=2 else np.nan

def last_3(series):
    return series.values[-3] if len(series.values)>=3 else np.nan

In [ ]:
#S_2是 計算每筆交易時間跟最後一筆交易時間間隔多久
agg_df =df.groupby('customer_ID').agg({'S_2':['last',last_2,last_3],'day_between_statment':['last',last_2,last_3]})
agg_df.columns = [i+'_'+j for i in ['S_2', 'days_between_statements'] for j in ['last', 'last_2', 'last_3']]

In [ ]:
df = df.groupby('customer_ID').first()
df = df.merge(agg_df,how='inner',left_index=True, right_index=True)
df = df.drop(['S_2','day_between_statment','S_2_first','S_2_last_x'],axis=1)
df.to_parquet('train_time_features.parquet')

## test

In [ ]:
df = pd.DataFrame(test_df[['customer_ID','S_2']],columns=['customer_ID','S_2'])
df['S_2'] = pd.to_datetime(df['S_2'])
df['day_count'] = df[['customer_ID','S_2']].groupby('customer_ID')['S_2'].transform('count')
df['S_2_first'] = df[['customer_ID','S_2']].groupby(['customer_ID'])['S_2'].transform('min')
df['S_2_last'] = df[['customer_ID','S_2']].groupby('customer_ID')['S_2'].transform('max')
df['S_2_period'] = (df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('max') - df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('min')).dt.days
df['day_between_statment'] = df[['customer_ID','S_2']].sort_values(['customer_ID','S_2']).groupby(['customer_ID'])['S_2'].transform('diff').dt.days
df['day_between_statment'] = df['day_between_statment'].fillna(0)

In [ ]:
df['day_between_statment_mean'] = df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('mean')
df['day_between_statment_std'] =df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('std')
df['day_between_statment_max'] =df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('max')
df['day_between_statment_min'] =df[['customer_ID','day_between_statment']].sort_values(['customer_ID','day_between_statment']).groupby('customer_ID')['day_between_statment'].transform('min')

In [ ]:
df['S_2'] = (df['S_2_last'] - df['S_2']).dt.days 
df['S_2_last_diff'] = (df['S_2_last'].max() - df['S_2_last']).dt.days
df['S_2_first_diff'] = (df['S_2_first'].min()-df['S_2_first']).dt.days

In [ ]:
df['S_2_first_dd'] = df['S_2_first'].dt.day
df['S_2_first_mm'] = df['S_2_first'].dt.month
df['S_2_first_yy'] = df['S_2_first'].dt.year

In [ ]:
df['S_2_last_dd'] = df['S_2_last'].dt.day
df['S_2_last_mm'] = df['S_2_last'].dt.month
df['S_2_last_yy'] = df['S_2_last'].dt.year

In [ ]:
agg_df =df.groupby('customer_ID').agg({'S_2':['last',last_2,last_3],'day_between_statment':['last',last_2,last_3]})
agg_df.columns = [i+'_'+j for i in ['S_2', 'days_between_statements'] for j in ['last', 'last_2', 'last_3']]
df = df.groupby('customer_ID').first()
df = df.merge(agg_df,how='inner',left_index=True, right_index=True)

In [ ]:
df = df.drop(['S_2','day_between_statment','S_2_first','S_2_last_x'],axis=1)
df.to_parquet('test_time_features.parquet')